In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.models.resnet import vgg_resnet50

import json
from glob import glob

In [5]:
torch.backends.cudnn.benchmark=True

## Data

In [6]:
PATH = Path('../data/all')

In [7]:
def show_img(im, figsize=None, ax=None, alpha=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    return ax

In [8]:
VEHICLES=10
ROADS=7
ROAD_LINES=6

In [9]:
TRAIN_DN = 'CameraRGB'
MASKS_DN = 'CameraSeg'
workers=7
random_crop=False
pseudo_label=False
val_folder = 'test'

In [10]:
S_PREFIX = '33-test_val'

In [11]:
from torchvision.datasets.folder import pil_loader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TTF

### Create dataloader

In [12]:
class MatchedFilesDataset(Dataset):
    def __init__(self, fnames, y, tfms, path):
        self.path,self.fnames = path,fnames
        self.open_fn = pil_loader
        self.y=y
        self.open_y_fn = pil_loader
        assert(len(fnames)==len(y))
        
        self.n = self.get_n()
        self.c = self.get_c()
        self.tfms = tfms
        
    def get_x(self, i): return self.open_fn(os.path.join(self.path, self.fnames[i]))
    def get_y(self, i): return self.open_y_fn(os.path.join(self.path, self.y[i]))
    def get_n(self): return len(self.fnames)
    def get_c(self): return 2
    
    def get(self, tfms, x, y):
        for fn in tfms:
            #pdb.set_trace()
            x, y = fn(x, y)
        return (x, y)
    
    def __getitem__(self, idx):
        x,y = self.get_x(idx),self.get_y(idx)
        return self.get(self.tfms, x, y)
    
    def __len__(self): return self.n

    def resize_imgs(self, targ, new_path):
        dest = resize_imgs(self.fnames, targ, self.path, new_path)
        return self.__class__(self.fnames, self.y, self.transform, dest)

In [13]:

# Seems to speed up training by ~2%
class DataPrefetcher():
    def __init__(self, loader, stop_after=None):
        self.loader = loader
        self.dataset = loader.dataset
        self.stream = torch.cuda.Stream()
        self.stop_after = stop_after
        self.next_input = None
        self.next_target = None

    def __len__(self):
        return len(self.loader)
    
    def preload(self):
        try:
            self.next_input, self.next_target = next(self.loaditer)
        except StopIteration:
            self.next_input = None
            self.next_target = None
            return
        with torch.cuda.stream(self.stream):
            self.next_input = self.next_input.cuda(async=True)
            self.next_target = self.next_target.cuda(async=True)

    def __iter__(self):
        count = 0
        self.loaditer = iter(self.loader)
        self.preload()
        while self.next_input is not None:
            torch.cuda.current_stream().wait_stream(self.stream)
            input = self.next_input
            target = self.next_target
            self.preload()
            count += 1
            yield input, target
            if type(self.stop_after) is int and (count > self.stop_after):
                break

In [14]:
def crop_bg_pil(x,y):
    w, h = x.size
    top = int(h/3.75)
    bot = int(h*.9 + h/150)
    pad_right=32-w%32
    return TTF.crop(x, top, 0, bot-top, w+pad_right), TTF.crop(y, top, 0, bot-top, w+pad_right)

In [15]:
class RHF(object):
    def __init__(self, p=0.5): self.p = p
    def __call__(self, x, y):
        if random.random() < self.p:
            return TTF.hflip(x), TTF.hflip(y)
        return x,y

In [16]:
class RR(object):
    def __init__(self, degrees=2): self.degrees = degrees
    def __call__(self, x, y):
        angle = random.uniform(-self.degrees, self.degrees)
        return TTF.rotate(x, angle), TTF.rotate(y, angle)

In [17]:
def tfm_x_wrapper(tfm):
    return lambda x,y: (tfm(x), y)

In [18]:
class RC():
    def __init__(self, targ_sz):
        self.targ_sz = targ_sz

    def __call__(self, x, y):
        rand_w = random.uniform(0, 1)
        rand_h = random.uniform(0, 1)
        w,h = x.size
        t_w,t_h = self.targ_sz
        start_x = np.floor(rand_w*(w-t_w)).astype(int)
        start_y = np.floor(rand_h*(h-t_h)).astype(int)
        return TTF.crop(x, start_y, start_x, t_h, t_w), TTF.crop(y, start_y, start_x, t_h, t_w)

In [19]:
def convert_y(y_img):
    yr = (y_img==ROADS) | (y_img==ROAD_LINES)
    yc = (y_img==VEHICLES)
    cutoff_y = int(yc.shape[0]*.87)
    yc[cutoff_y:,:] = 0
    return torch.from_numpy(np.stack((yc,yr)).astype(int))


def xy_tensor(x,y):
    y_img = np.array(y, np.int32, copy=False)
    return TTF.to_tensor(x), convert_y(y_img[:,:,0])

In [20]:
class RRC(transforms.RandomResizedCrop):
    def __call__(self, x, y):
        i, j, h, w = self.get_params(x, self.scale, self.ratio)
        x = TTF.resized_crop(x, i, j, h, w, self.size, self.interpolation)
        y = TTF.resized_crop(y, i, j, h, w, self.size, self.interpolation)
        return x, y

In [21]:
def torch_loader(f_ext, data_path, bs, size, workers=7, random_crop=False, pseudo_label=False, val_folder=None):
    # Data loading code
    x_names = np.sort(np.array(glob(str(data_path/f'CameraRGB{f_ext}'/'*.png'))))
    y_names = np.sort(np.array(glob(str(data_path/f'CameraSeg{f_ext}'/'*.png'))))

    x_n = x_names.shape[0]
    val_idxs = list(range(x_n-300, x_n))
    
    if pseudo_label:
        x_names_test = np.sort(np.array(glob(f'../data/pseudo/CameraRGB{f_ext}/*.png')))
        y_names_test = np.sort(np.array(glob(f'../data/pseudo/CameraSeg{f_ext}/*.png')))
        x_names = np.concatenate((x_names, x_names_test))
        x_names = np.concatenate((y_names, y_names_test))
        print(f'Pseudo-Labels: {len(x_names_test)}')
    if val_folder:
        x_names_val = np.sort(np.array(glob(f'../data/{val_folder}/CameraRGB{f_ext}/*.png')))
        y_names_val = np.sort(np.array(glob(f'../data/{val_folder}/CameraSeg{f_ext}/*.png')))
        val_x,val_y = x_names_val, y_names_val
        trn_x,trn_y = x_names, y_names
        print(f'Val Labels:', len(val_x))
    else:
        ((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, x_names, y_names)
    print(f'Val x:{len(val_x)}, y:{len(val_y)}')
    print(f'Trn x:{len(trn_x)}, y:{len(trn_y)}')
    print(f'All x:{len(x_names)}')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    train_tfms = [
        crop_bg_pil,
        tfm_x_wrapper(transforms.ColorJitter(.2,.2,.2)),
#         tfm_x_wrapper(Lighting(0.1, __imagenet_pca['eigval'], __imagenet_pca['eigvec'])),
        RR(),
        RHF(),
#         RC((size,size)),
        xy_tensor,
        tfm_x_wrapper(normalize),
    ]
    if random_crop:
        train_tfms.insert(3,RRC(size, scale=(0.4, 1.0)))
    train_dataset = MatchedFilesDataset(trn_x, trn_y, train_tfms, path='')
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_tfms = [
        crop_bg_pil,
        xy_tensor,
        tfm_x_wrapper(normalize)
    ]
    val_dataset = MatchedFilesDataset(val_x, val_y, val_tfms, path='')
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs, shuffle=False,
        num_workers=workers, pin_memory=True)

    train_loader = DataPrefetcher(train_loader)
    val_loader = DataPrefetcher(val_loader)
    
    data = ModelData(data_path, train_loader, val_loader)
    return data


In [22]:
def denorm(x):
    x_np = x.cpu().numpy()
    x_np = np.rollaxis(x_np, 0, 3)
    mean=np.array([0.4914 , 0.48216, 0.44653])
    std=np.array([0.24703, 0.24349, 0.26159])
    x_np = x_np*std+mean
    return x_np

## U-net (ish)

In [23]:
from torchvision.models import vgg11_bn

In [24]:
def vgg11(pre): return children(vgg11_bn(pre))[0]

In [25]:
model_meta = {
    resnet18:[8,6], resnet34:[8,6], resnet50:[8,6], resnet101:[8,6], resnet152:[8,6],
    vgg11:[0,13], vgg16:[0,22], vgg19:[0,22],
    resnext50:[8,6], resnext101:[8,6], resnext101_64:[8,6],
    wrn:[8,6], inceptionresnet_2:[-2,9], inception_4:[-1,9],
    dn121:[0,7], dn161:[0,7], dn169:[0,7], dn201:[0,7],
}

In [26]:
def get_base(f):
    cut,lr_cut = model_meta[f]
    layers = cut_model(f(True), cut)
    return nn.Sequential(*layers), lr_cut

In [27]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [48]:
BN_EPS = 1e-4  #1e-4  #1e-5
class ConvBnRelu2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, dilation=1, stride=1, groups=1, is_bn=True, is_relu=True):
        super(ConvBnRelu2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation, groups=groups, bias=False)
        self.bn   = nn.BatchNorm2d(out_channels, eps=BN_EPS)
        self.relu = nn.ReLU(inplace=True)
        if is_bn   is False: self.bn  =None
        if is_relu is False: self.relu=None


    def forward(self,x):
        x = self.conv(x)
        if self.bn   is not None: x = self.bn(x)
        if self.relu is not None: x = self.relu(x)
        return x
    
class ConvResidual (nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ConvResidual, self).__init__()

        self.block = nn.Sequential(
            ConvBnRelu2d(in_channels,  out_channels, kernel_size=3, padding=1,  stride=1 ),
            ConvBnRelu2d(out_channels, out_channels, kernel_size=3, padding=1,  stride=1, is_relu=False),
        )
        self.shortcut = None
        if in_channels!=out_channels or stride!=1:
            self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, stride=stride,  bias=True)

    def forward(self, x):
        r = x if self.shortcut is None else self.shortcut(x)
        x = self.block(x)
        x = F.relu(x.add_(r), inplace=True)
        return x
    

## origainl 3x3 stack filters used in UNet## origa 
class StackEncoder (nn.Module):
    def __init__(self, x_channels, y_channels, kernel_size=3, max_pool=False):
        super(StackEncoder, self).__init__()
        padding=(kernel_size-1)//2
        self.encode = nn.Sequential(
            ConvBnRelu2d(x_channels, y_channels, kernel_size=kernel_size, padding=padding, dilation=1, stride=1, groups=1),
            ConvBnRelu2d(y_channels, y_channels, kernel_size=kernel_size, padding=padding, dilation=1, stride=1, groups=1),
        )
        self.mp = nn.MaxPool2d(2) if max_pool else nn.Conv2d(y_channels, y_channels, kernel_size=kernel_size, padding=padding, stride=2)

    def forward(self,x):
        y = self.encode(x)
        y_small = self.mp(y)
        return y, y_small


class StackDecoder (nn.Module):
    def __init__(self, x_big_channels, x_channels, y_channels, kernel_size=3):
        super(StackDecoder, self).__init__()
        padding=(kernel_size-1)//2

        self.decode = nn.Sequential(
            ConvBnRelu2d(x_big_channels+x_channels, y_channels, kernel_size=kernel_size, padding=padding, dilation=1, stride=1, groups=1),
            ConvBnRelu2d(y_channels, y_channels, kernel_size=kernel_size, padding=padding, dilation=1, stride=1, groups=1),
            ConvBnRelu2d(y_channels, y_channels, kernel_size=kernel_size, padding=padding, dilation=1, stride=1, groups=1),
        )

    def forward(self, x_big, x):
        N,C,H,W = x_big.size()
        y = F.upsample(x, size=(H,W),mode='bilinear')
        #y = F.upsample(x, scale_factor=2,mode='bilinear')
        y = torch.cat([y,x_big],1)
        y = self.decode(y)
        return  y
    # 256x256
    
class Unet256 (nn.Module):
    def __init__(self, out_c=2, in_c=3, f=2):
        super().__init__()
#         C,H,W = in_shape
        #assert(C==3)

        #256
        self.down2 = StackEncoder(in_c,   64//f, kernel_size=3)   #128
        self.down3 = StackEncoder( 64//f,  128//f, kernel_size=3)   # 64
        self.down4 = StackEncoder(128//f,  256//f, kernel_size=3)   # 32
        self.down5 = StackEncoder(256//f,  512//f, kernel_size=3)   # 16
        self.down6 = StackEncoder(512//f, 1024//f, kernel_size=3)   #  8

        self.center = nn.Sequential(
            #ConvBnRelu2d( 512, 1024, kernel_size=3, padding=1, stride=1 ),
            ConvBnRelu2d(1024//f, 1024//f, kernel_size=3, padding=1, stride=1 ),
        )

        # 8
        # x_big_channels, x_channels, y_channels
        self.up6 = StackDecoder(1024//f,1024//f, 512//f, kernel_size=3)  # 16
        self.up5 = StackDecoder( 512//f, 512//f, 256//f, kernel_size=3)  # 32
        self.up4 = StackDecoder( 256//f, 256//f, 128//f, kernel_size=3)  # 64
        self.up3 = StackDecoder( 128//f, 128//f,  64//f, kernel_size=3)  #128
        self.up2 = StackDecoder(  64//f,  64//f,  32, kernel_size=3)  #256
        self.classify = nn.Conv2d(32, out_c, kernel_size=1, padding=0, stride=1, bias=True)


    def forward(self, x):

        out = x                       #;print('x    ',x.size())
                                      #
        down2,out = self.down2(out)   #;print('down2',down2.size())  #128
        down3,out = self.down3(out)   #;print('down3',down3.size())  #64
        down4,out = self.down4(out)   #;print('down4',down4.size())  #32
        down5,out = self.down5(out)   #;print('down5',down5.size())  #16
        down6,out = self.down6(out)   #;print('down6',down6.size())  #8
        pass                          #;print('out  ',out.size())

        out = self.center(out)
        out = self.up6(down6, out)
        out = self.up5(down5, out)
        out = self.up4(down4, out)
        out = self.up3(down3, out)
        out = self.up2(down2, out)

        out = self.classify(out)
        out = torch.squeeze(out, dim=1)
        return out

In [35]:
class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p, inplace=True))

In [36]:
class Unet34(nn.Module):
    def __init__(self, f=resnet34):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5 = UnetBlock(256,3,16)
        self.up6 = nn.ConvTranspose2d(16, 2, 1)
        
    def forward(self,x):
        inp = x
        x = F.relu(self.rn(x), inplace=True)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x, inp)
        x = self.up6(x)
        return torch.squeeze(x)
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [37]:
class Unet34Mod(nn.Module):
    def __init__(self, f=resnet34):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5 = UnetBlock(256,16,16)
        self.up6 = nn.ConvTranspose2d(16, 2, 1)
        self.x_skip = nn.Sequential(
            nn.Conv2d(3,16,1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        
    def forward(self,x):
        x_skip = self.x_skip(x)
        x = F.relu(self.rn(x), inplace=True)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x, x_skip)
        x = self.up6(x)
        return torch.squeeze(x)
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [51]:
class Unet11(nn.Module):
    def __init__(self, f=vgg11):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[0][i]) for i in [2,6,13,20,27]]
        self.up0 = UnetBlock(512,512,256)
        self.up1 = UnetBlock(256,512,256)
        self.up2 = UnetBlock(256,256,256)
        self.up3 = UnetBlock(256,128,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5  = nn.Conv2d(256,2,1)
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up0(x, self.sfs[4].features)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x)
        return x

In [39]:
class UnetModel():
    def __init__(self,model,name='unet'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        if isinstance(self.model, FP16):
            model = self.model.module
        else:
            model = self.model
        lgs = list(split_by_idxs(children(model.rn), [model.lr_cut]))
#         print('LGS:', lgs)
#         print('Add:', children(model)[1:])
        return lgs + [children(model)[1:]]

In [40]:
def car_f_p_r(pred, targs):
    p2 = F.sigmoid(pred)
    return fbeta_score(targs[:,0,:,:], p2[:,0,:,:], beta=2, threshold=0.5)

In [41]:
def rd_f(pred, targs):
    mx,idx = torch.max(pred, 1)
    p2 = F.sigmoid(pred)
    f,p,r = fbeta_score(targs[:,1,:,:], p2[:,1,:,:], beta=0.5, threshold=0.5)
    return f

In [42]:
def fbeta_score(y_true, y_pred, beta, threshold=None, eps=1e-9):
    beta2 = beta**2

    if threshold:
        y_pred = torch.ge(y_pred.float(), threshold).float()
    else:
        y_pred = y_pred.float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum()
    precision = true_positive/(y_pred.sum()+(eps))
    recall = true_positive/(y_true.sum()+eps)
    
    fb = (precision*recall)/(precision*beta2 + recall + eps)*(1+beta2)
    
    return fb, precision, recall

In [43]:
def new_acc(pred, targs):
    p2 = F.sigmoid(pred)
    return ((p2>0.5).long() == targs).float().mean()
def dice_mult(pred, targs):
#     pred = (pred>0).float()
    mx,idx = torch.max(pred, 1)
    pred = idx.float()
    targs = targs.float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()
def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()

In [44]:
def dice_coeff_weight(pred, target, weight):
    smooth = 1.
    num,c,h,w = pred.shape
    m1 = pred.view(num, c, -1)  # Flatten
    m2 = target.view(num, c, -1)  # Flatten
    intersection = (m1 * m2)
    w = V(torch.cuda.FloatTensor(weight).view(1,-1,1))
    i_w = (w*intersection).sum()
    m1_w = (w*m1).sum()
    m2_w = (w*m2).sum()
    return (2. * i_w + smooth) / (m1_w + m2_w + smooth)

def dice_coeff(pred, target):
    smooth = 1.
    num,c,h,w = pred.shape
    m1 = pred.view(num, c, -1)  # Flatten
    m2 = target.view(num, c, -1)  # Flatten
    intersection = (m1 * m2).sum()
    return (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)


class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()
        self.weight = weight

    def forward(self, logits, targets):
        probs = F.sigmoid(logits)
        num = targets.size(0)  # Number of batches

        if self.weight:
            score = dice_coeff_weight(probs, targets.float(), self.weight)
        else:
            score = dice_coeff(probs, targets.float())
        score = 1 - score.sum() / num
        return score

In [45]:
def get_learner(md, m_fn=Unet34):
    m = to_gpu(m_fn())
    models = UnetModel(m)
    learn = ConvLearner(md, models)
    learn.opt_fn=optim.Adam
#     class_weights = torch.FloatTensor([1,3,2]).cuda()
#     learn.crit=nn.CrossEntropyLoss(weight=class_weights)
    learn.crit=SoftDiceLoss(weight=[1,1/100])
    learn.metrics=[new_acc, rd_f, car_f_p_r]
    return learn

### Unet34

In [46]:
ext = '-150'
sz = 96
bs = 128
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label)

learn = get_learner(md, m_fn=Unet34)

Val x:300, y:300
Trn x:6980, y:6980
All x:7280


In [47]:
lr=1e-1
wd=1e-5
learn.fit(lr,1,wds=wd,cycle_len=15,use_clr=(5,8))

epoch      trn_loss   val_loss   new_acc    rd_f       car_f_p_r  
    0      0.995757   0.998675   0.649951   0.381529   0.274048   0.076501   0.874585  
    1      0.99622    0.996088   0.873326   0.565419   0.252827   0.670706   0.223495  
    2      0.995634   0.997873   0.720794   0.403747   0.285335   0.133232   0.430019  
    3      0.995147   0.999536   0.712422   0.207848   0.112715   0.029258   0.448258  
    4      0.995      0.995697   0.896916   0.669429   0.531416   0.466303   0.577838  
    5      0.995155   0.994137   0.777962   0.442009   0.589179   0.700061   0.567799  
    6      0.994823   0.995838   0.886459   0.632443   0.648728   0.393139   0.829484  
    7      0.99491    0.997384   0.840897   0.109644   0.528947   0.240293   0.820316  
    8      0.994867   0.994801   0.827438   0.505419   0.49205    0.612135   0.478694  
    9      0.994765   0.999592   0.661632   0.393339   0.090181   0.021898   0.470762  
    10     0.99461    0.99674    0.851195   0.000119 

[array([0.99279]),
 0.9273323551813761,
 0.7435713576810119,
 0.6911577680491728,
 0.8459216177691435,
 0.6618729190709222]

### Unet34-Mod

In [49]:
ext = '-150'
sz = 96
bs = 128
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label)

learn = get_learner(md, m_fn=Unet34Mod)

Val x:300, y:300
Trn x:6980, y:6980
All x:7280


In [50]:
lr=1e-1
wd=1e-5
learn.fit(lr,1,wds=wd,cycle_len=15,use_clr=(5,8))

epoch      trn_loss   val_loss   new_acc    rd_f       car_f_p_r  
    0      0.995955   0.997948   0.70653    0.411775   0.395894   0.130579   0.901688  
    1      0.996291   0.995748   0.820317   0.503778   0.28063    0.45092    0.259412  
    2      0.995906   0.997025   0.738731   0.354821   0.366208   0.330307   0.405448  
    3      0.995363   0.999387   0.779061   0.597251   0.114889   0.030124   0.433363  
    4      0.995096   0.995426   0.884654   0.61745    0.390542   0.58222    0.367978  
    5      0.995088   0.998242   0.666772   0.376902   0.362389   0.122178   0.78331   
    6      0.995085   0.994677   0.897554   0.649635   0.32618    0.820497   0.284132  
    7      0.995203   0.994761   0.863264   0.555144   0.523495   0.661163   0.503039  
    8      0.995072   0.995295   0.867916   0.577368   0.317151   0.769341   0.276839  
    9      0.994821   0.995292   0.897009   0.649539   0.239248   0.860685   0.203745  
    10     0.994579   0.994327   0.891842   0.629439 

[array([0.99668]),
 0.8909398976961772,
 0.6648937471712361,
 0.5037837062397947,
 0.2666611634567854,
 0.7047605882565942]

### Unet256

In [55]:
def get_unet256_learner(md, m_fn):
    m = m_fn(f=2)
    learn = Learner.from_model_data(m, md)
    learn.opt_fn=optim.Adam
#     class_weights = torch.FloatTensor([1,3,2]).cuda()
#     learn.crit=nn.CrossEntropyLoss(weight=class_weights)
    learn.crit=SoftDiceLoss(weight=[1,1/100])
    learn.metrics=[new_acc, rd_f, car_f_p_r]
    return learn

In [56]:
ext = '-150'
sz = 96
bs = 128
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label)

learn = get_unet256_learner(md, m_fn=Unet256)

Val x:300, y:300
Trn x:6980, y:6980
All x:7280


In [57]:
lr=1e-1
wd=1e-5
learn.fit(lr,1,wds=wd,cycle_len=15,use_clr=(5,8))

epoch      trn_loss   val_loss   new_acc    rd_f       car_f_p_r  
    0      0.99759    0.999496   0.549292   0.329369   0.064396   0.017711   0.21505   
    1      0.997147   0.999464   0.581651   0.329369   0.020531   0.006625   0.06362   
    2      0.997041   0.99787    0.633898   0.329369   0.004846   0.27836    0.00389   
    3      0.996982   0.999031   0.591616   0.329369   0.111669   0.031427   0.316671  
    4      0.996901   0.999599   0.31975    0.329369   0.092622   0.020423   0.898683  
    5      0.996826   0.99735    0.633967   0.329369   0.014767   0.336428   0.011918  
    6      0.997002   0.997492   0.633838   0.329369   0.0        0.0        0.0       
    7      0.99706    0.997468   0.633882   0.329369   0.0        0.0        0.0       
    8      0.997068   0.997491   0.633833   0.329419   0.0        0.0        0.0       
    9      0.997058   0.997489   0.633851   0.32942    0.0        0.0        0.0       
    10     0.997041   0.997534   0.63384    0.329431 

[array([0.99749]), 0.6338796544075013, 0.329368738706921, 0.0, 0.0, 0.0]

### Unet11

In [52]:
ext = '-150'
sz = 96
bs = 128
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label)

learn = get_learner(md, m_fn=Unet11)

Val x:300, y:300
Trn x:6980, y:6980
All x:7280


In [53]:
lr=1e-1
wd=1e-5
learn.fit(lr,1,wds=wd,cycle_len=15,use_clr=(5,8))

epoch      trn_loss   val_loss   new_acc    rd_f       car_f_p_r  
    0      0.995109   0.999037   0.692892   0.442539   0.212356   0.054974   0.829949  
    1      0.995092   0.999455   0.645618   0.494868   0.113549   0.026868   0.674744  
    2      0.994787   0.994858   0.926342   0.775052   0.525516   0.540308   0.533218  
    3      0.994555   0.994226   0.938998   0.78364    0.671442   0.566461   0.728374  
    4      0.994347   0.994266   0.78847    0.457138   0.483492   0.873292   0.436065  
    5      0.994245   0.994151   0.935164   0.769386   0.548665   0.612113   0.545308  
    6      0.994113   0.993785   0.938096   0.80512    0.452985   0.887821   0.403618  
    7      0.994241   0.99337    0.936344   0.856293   0.593633   0.854688   0.552496  
    8      0.994091   0.993212   0.965152   0.881042   0.722872   0.634158   0.767674  
    9      0.994166   0.993583   0.936696   0.796099   0.628609   0.696393   0.621932  
    10     0.994051   0.993655   0.957059   0.836327 

[array([0.99214]),
 0.9792524631818136,
 0.9275796857288896,
 0.6571543484412488,
 0.8825794208929445,
 0.6179412810897752]

### Unet11-Mini

In [60]:
test = vgg11(pre=False)

In [61]:
test

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (6): ReLU(inplace)
  (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (10): ReLU(inplace)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (13): ReLU(inplace)
  (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (16): BatchNorm2d(51

In [62]:
class Unet11Mini(nn.Module):
    def __init__(self, f=vgg11):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[0][i]) for i in [2,6,13,20,27]]
        self.up0 = UnetBlock(512,512,256)
        self.up1 = UnetBlock(256,512,256)
        self.up2 = UnetBlock(256,256,256)
        self.up3 = UnetBlock(256,128,128)
        self.up4 = UnetBlock(128,64,64)
        self.up5  = nn.Conv2d(64,2,1)
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up0(x, self.sfs[4].features)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x)
        return x

In [63]:
ext = '-150'
sz = 96
bs = 128
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label)

learn = get_learner(md, m_fn=Unet11Mini)

Val x:300, y:300
Trn x:6980, y:6980
All x:7280


In [64]:
lr=1e-1
wd=1e-5
learn.fit(lr,1,wds=wd,cycle_len=15,use_clr=(5,8))

epoch      trn_loss   val_loss   new_acc    rd_f       car_f_p_r  
    0      0.995271   0.999623   0.578141   0.620581   0.099298   0.021895   0.979557  
    1      0.994763   0.999042   0.766232   0.599266   0.207313   0.052537   0.885654  
    2      0.994749   0.993679   0.87984    0.602259   0.560635   0.749528   0.528512  
    3      0.994441   0.996227   0.928851   0.79984    0.567761   0.287903   0.808903  
    4      0.994496   0.993823   0.935322   0.787189   0.489986   0.800444   0.448098  
    5      0.994293   0.993857   0.906809   0.668762   0.479968   0.886636   0.432401  
    6      0.994309   0.993882   0.911666   0.687226   0.702128   0.604251   0.754536  
    7      0.994552   0.997677   0.903929   0.721601   0.290022   0.13701    0.429104  
    8      0.994537   0.9944     0.927072   0.735713   0.323486   0.884108   0.279818  
    9      0.994392   0.993301   0.937605   0.771508   0.577911   0.779382   0.543697  
    10     0.994221   0.993344   0.937727   0.771228 

[array([0.99295]),
 0.9390928109486898,
 0.7748000189510555,
 0.74791829254722,
 0.703414797815009,
 0.7702982393883164]

### Unet34-ModWide

In [69]:
class Unet34ModWide(nn.Module):
    def __init__(self, f=resnet34):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5 = UnetBlock(256,32,32)
        self.up6 = nn.ConvTranspose2d(32, 2, 1)
        self.x_skip = nn.Sequential(
            nn.Conv2d(3,32,1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        
    def forward(self,x):
        x_skip = self.x_skip(x)
        x = F.relu(self.rn(x), inplace=True)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x, x_skip)
        x = self.up6(x)
        return torch.squeeze(x)
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [70]:
ext = '-150'
sz = 96
bs = 128
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label)

learn = get_learner(md, m_fn=Unet34ModWide)

Val x:300, y:300
Trn x:6980, y:6980
All x:7280


In [71]:
lr=1e-1
wd=1e-5
learn.fit(lr,1,wds=wd,cycle_len=15,use_clr=(5,8))

epoch      trn_loss   val_loss   new_acc    rd_f       car_f_p_r  
    0      0.995684   0.999085   0.732984   0.51987    0.214276   0.05399    0.938878  
    1      0.996277   0.998894   0.837445   0.565755   0.063257   0.031844   0.087394  
    2      0.996596   0.996335   0.791452   0.465972   0.129097   0.491905   0.110321  
    3      0.995704   0.997055   0.863653   0.60141    0.477726   0.219044   0.740917  
    4      0.995041   0.997293   0.867035   0.62005    0.43503    0.187021   0.713084  
    5      0.994923   0.993695   0.846692   0.542982   0.622922   0.714568   0.604308  
    6      0.994846   0.994169   0.894069   0.638061   0.569111   0.612374   0.565585  
    7      0.994586   0.994805   0.88764    0.612637   0.665314   0.593014   0.703626  
    8      0.994687   0.994993   0.884644   0.623509   0.647842   0.438031   0.773144  
    9      0.994793   0.99424    0.875201   0.592203   0.498725   0.77367    0.458743  
    10     0.994775   0.994664   0.890124   0.634241 

[array([0.994]),
 0.8734409904479981,
 0.5965213615032877,
 0.7203751306039818,
 0.5449415823947977,
 0.809040420483691]